In [2]:
from imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from hit_makers.biasgan_hit_maker import BiasganHitMakerRace
from hit_makers.biasgan_hit_maker import task_to_class

In [4]:
task_to_size_of_chunk = {
    "Task1_Race_Classification": 120,
    "Task2_Real_Fake_Classification": 145,
    "Task3_Quality_Ranking": 100,
}

In [16]:
directory = "biasgan/mturk_data"
task1 = "Task1_Race_Classification"
task2 = "Task2_Real_Fake_Classification"
task3 = "Task3_Quality_Ranking"

task = task3
# NOTE(ethan): task 3 data folder seems incorrect right now
maker = task_to_class[task]()

task_folder = goat.pjoin(directory, task)
print(task_folder)

# from hit_makers.biasgan_hit_maker import get_processed_data_from_task_folder
pdata = maker.get_processed_data_from_media_folder(task_folder)
# pprint.pprint(pdata)

biasgan/mturk_data/Task3_Quality_Ranking


In [17]:
pdata.keys()

dict_keys(['task', 'task_question', 'examples', 'gt_query_examples', 'query_folder_to_query_examples', 'query_folder_query_folder_to_query_examples'])

In [18]:
# pdata["query_folder_to_query_examples"]

In [19]:
# pdata["gt_query_examples"]

In [22]:
# now, break the save a bunch of hits (e.g., 40)...
# 100 images per HIT
# chunk with goat
# size_of_chunk = 100
size_of_chunk = task_to_size_of_chunk[task]
print(size_of_chunk)

query_folder_to_query_examples = pdata["query_folder_to_query_examples"]
for query_folder in query_folder_to_query_examples:
    query_examples = query_folder_to_query_examples[query_folder]
    random.shuffle(query_examples)
    query_examples_chunks = goat.utils.get_chunks(query_examples, size_of_chunk=size_of_chunk)
    for hit_idx, qes in enumerate(query_examples_chunks):
        examples = pdata["examples"]
        gt_query_examples = random.sample(pdata["gt_query_examples"], k=10)
        hit = maker.get_hit(task=task, 
                            task_question=pdata["task_question"], 
                            examples=examples, 
                            gt_query_examples=gt_query_examples, 
                            query_examples=qes)
        hit_name = "{}-{}-{:06d}".format(task, query_folder, hit_idx)
#         print(hit_name)
        maker.save(hit_name, hit)
    
if "query_folder_query_folder_to_query_examples" in pdata: # for inter splits
    query_folder_query_folder_to_query_examples = pdata["query_folder_query_folder_to_query_examples"]
    for query_folder_query_folder in query_folder_query_folder_to_query_examples:
        query_examples = query_folder_query_folder_to_query_examples[query_folder_query_folder]
        random.shuffle(query_examples)
        query_examples_chunks = goat.utils.get_chunks(query_examples, size_of_chunk=size_of_chunk)
        for hit_idx, qes in enumerate(query_examples_chunks):
            examples = pdata["examples"]
            gt_query_examples = random.sample(pdata["gt_query_examples"], k=10)
            hit = maker.get_hit(task=task, 
                                task_question=pdata["task_question"], 
                                examples=examples, 
                                gt_query_examples=gt_query_examples, 
                                query_examples=qes)
            hit_name = "{}-{}-{:06d}".format(task, query_folder_query_folder, hit_idx)
    #         print(hit_name)
            maker.save(hit_name, hit)

100


In [23]:
pdata.keys()

dict_keys(['task', 'task_question', 'examples', 'gt_query_examples', 'query_folder_to_query_examples', 'query_folder_query_folder_to_query_examples'])

In [24]:
# "/shared/ethanweber/anno/static/data/hits"
interface_endpoint = "https://biasgan.ethanweber.me/interface"
hit_names = os.listdir("/shared/ethanweber/anno/static/data/hits/")
external_urls = []
for hit_name in sorted(hit_names):
    external_urls.append(os.path.join(interface_endpoint, hit_name.replace(".json", "")))
# external_urls

In [25]:
len(external_urls)
# external_urls

474

In [34]:
external_urls = set(external_urls) - set(temp)

In [37]:
len(external_urls) + len(temp)

474

In [39]:
# len(external_urls)

In [44]:
# sorted(external_urls)

In [42]:
# GLOBAL_REAL_TEST_TIME

In [34]:
dolors_per_hour = 9
minutes_per_hit = 5
cost_per_hit = (minutes_per_hit / 60) * dolors_per_hour
num_hits = len(external_urls)
mturk_tax = 1.4
num_assignments_per_hit = 3
cost = cost_per_hit * num_hits * num_assignments_per_hit * mturk_tax
print(cost_per_hit)
print(cost)

NameError: name 'external_urls' is not defined

In [288]:
# 100 * 5.4

In [289]:
# hit_names

In [33]:
cost_per_hit



NameError: name 'cost_per_hit' is not defined

In [1]:
# examples = [example] * 10
# hit = maker.get_hit(examples, examples, examples)
# pprint.pprint(hit)
# hit_name = "hit_example"
# maker.save(hit_name, hit)

In [51]:
# external_urls

In [7]:
from requester import Requester

In [8]:
use_sandbox=False
DATABASE_FILENAME = "/shared/ethanweber/anno/mturk_database.pkl"
CREDS_FILENAME = "/shared/ethanweber/anno/mturk_creds.json"

requester = Requester(use_sandbox=use_sandbox,
                      database_filename=DATABASE_FILENAME,
                      creds_filename=CREDS_FILENAME)
requester.show_account_balance()

233.41


In [50]:
total_num_questions = 0 # includes quality control
total_time = 0
num_questions = 0
num_hits = 0
total_cost = 0
for hit_id, v in requester.database["production"].items():
    for r in v["response"]:
        num_questions += len(r["QUERY_EXAMPLES_RESPONSES"])
        total_num_questions += len(r["GLOBAL_ALL_IDS"])
        total_time += r["GLOBAL_REAL_TEST_TIME"]
        num_hits += 1
        total_cost += 0.75
num_questions /= 3

average_time_per_question = total_time / total_num_questions
print(average_time_per_question)

total_time_in_hours = total_time / (60*60)

2.4710681517798654


In [51]:
# total_time_in_hours

122.48261138988866

In [45]:
total_cost

1066.5

In [49]:
# total_time

In [52]:
total_cost / total_time_in_hours

8.707358439681693

In [48]:
total_cost / (total_time / (60*60)

0.002418710677689359

In [46]:
total_time / total_cost

413.4434139743077

In [40]:
total_cost

0

In [38]:
total_time

440937.40100359917

In [35]:
hours / question

NameError: name 'hours' is not defined

In [30]:
total_num_questions / 3

59480.0

In [28]:
r.keys()

dict_keys(['GLOBAL_ALL_IDS', 'GLOBAL_ALL_INDICES', 'GLOBAL_CONFIG_NAME', 'GLOBAL_ALL_RESPONSES', 'GT_HIDDEN_EXAMPLES_RESPONSES', 'GT_HIDDEN_EXAMPLES_RESPONSES_REVERSED', 'QUERY_EXAMPLES_RESPONSES', 'GLOBAL_REAL_TEST_TIME', 'GLOBAL_CONFIG', 'AssignmentId', 'WorkerId'])

In [19]:
v["response"][0]["QUERY_EXAMPLES_RESPONSES"]

['Cannot Determine',
 'White',
 'Cannot Determine',
 'Cannot Determine',
 'Cannot Determine',
 'Cannot Determine',
 'Cannot Determine',
 'Black',
 'Non-black/white',
 'White',
 'Black',
 'White',
 'Black',
 'Non-black/white',
 'Cannot Determine',
 'White',
 'Non-black/white',
 'Cannot Determine',
 'Non-black/white',
 'Cannot Determine',
 'Non-black/white',
 'Non-black/white',
 'Black',
 'Black',
 'Black',
 'Non-black/white',
 'Cannot Determine',
 'Cannot Determine',
 'Black',
 'Black',
 'Non-black/white',
 'Non-black/white',
 'Cannot Determine',
 'Non-black/white',
 'Non-black/white',
 'Non-black/white',
 'Non-black/white',
 'Non-black/white',
 'Non-black/white',
 'White',
 'Cannot Determine',
 'Cannot Determine',
 'White',
 'Black',
 'Cannot Determine',
 'White',
 'Black',
 'Non-black/white',
 'Cannot Determine',
 'White',
 'Non-black/white',
 'Cannot Determine',
 'Cannot Determine',
 'Cannot Determine',
 'Cannot Determine',
 'White',
 'Cannot Determine',
 'Non-black/white',
 'Cannot 

In [ ]:
1570.09

In [337]:
1570.09 - 800.23

769.8599999999999

In [48]:
lifetime_hours = 12
per_hit_minutes = 15
cost_per_hit = '0.75'
days_until_autoapprove = 2
estimated_time_in_minutes = 5
task_attributes = {
    # Number of assignments per task.
    'MaxAssignments': 3,
    # How long the task will be available on MTurk
    'LifetimeInSeconds': 60 * 60 * lifetime_hours,
    # How long Workers have to complete each item
    'AssignmentDurationInSeconds': 60 * per_hit_minutes,
    # The reward you will offer Workers for each response
    'Reward': cost_per_hit,
    'Title': 'Identify the race of person(s) or quality in the image. (~{} min)'.format(estimated_time_in_minutes),
    'Keywords': 'image, choice, computer vison, selection',
    'Description': 'Identify the race of person(s) or quality in the image.',
    'AutoApprovalDelayInSeconds': days_until_autoapprove * (60*60*24)
}
requester.set_task_attributes(task_attributes)

In [46]:
requester.get_worker_requirements()

[{'QualificationTypeId': '2F1QJWKUDD8XADTFD2Q0G6UTO95ALH',
  'Comparator': 'Exists'}]

In [302]:
# task1_external_urls = ["https://biasgan.ethanweber.me/interface/Task1_Race_Classification-black20_white80-000000"]


In [162]:
# https://biasgan.ethanweber.me/interface/Task1_Race_Classification-black20_white80-000000
# https://biasgan.ethanweber.me/interface/Task2_Real_Fake_Classification-black20_white80-000000
# https://biasgan.ethanweber.me/interface/Task3_Quality_Ranking-black20_white80-000000

In [163]:
# let's run the first 10 of each

In [303]:
# urls = []
# for external_url in external_urls:
#     if "000000" in external_url and external_url.find("black20_white80") < 0:
#         urls.append(external_url)

In [165]:
urls

['https://biasgan.ethanweber.me/interface/Task1_Race_Classification-black50_white50-000000',
 'https://biasgan.ethanweber.me/interface/Task1_Race_Classification-black80_white20-000000',
 'https://biasgan.ethanweber.me/interface/Task1_Race_Classification-fface_val_data-000000',
 'https://biasgan.ethanweber.me/interface/Task2_Real_Fake_Classification-black50_white50-000000',
 'https://biasgan.ethanweber.me/interface/Task2_Real_Fake_Classification-black80_white20-000000',
 'https://biasgan.ethanweber.me/interface/Task3_Quality_Ranking-black50_white50-000000',
 'https://biasgan.ethanweber.me/interface/Task3_Quality_Ranking-black80_white20-000000']

In [166]:
# external_urls

In [338]:
# external_urls

In [305]:
# TODO(ethan): add a catch to be careful not to resubmit the same HIT...
hit_ids = requester.submit_hits_from_external_urls(external_urls)

100%|██████████| 273/273 [01:54<00:00,  2.39it/s]


In [47]:
hit_ids = requester.submit_hits_from_external_urls(external_urls)

100%|██████████| 201/201 [02:48<00:00,  1.19it/s]


In [307]:
len(hit_ids)

273

In [31]:
temp = []
for hit_id in list(requester.database["production"].keys()):
    temp.append(requester.database["production"][hit_id]["external_url"])

In [33]:
len(temp)

273

In [28]:
temp

[]

In [313]:
# requester.write_database()

In [100]:
hit_ids

['3PUV2Q8SV4YBG0Q42LXUWOT1G20DBQ']

In [204]:
hit_ids = [v["hit_id"] for v in requester.database["production"].values()]

In [205]:
hit_ids

['3E22YV8GG10M66IRAKTYNDAKBAHPN5',
 '3PUV2Q8SV4YBG0Q42LXUWOT1G20DBQ',
 '33P2GD6NRNMTLJZ0UUTDMCRPVO3KHR',
 '3G3AJKPCXLMUTC0MJ2AL1VXWOYQ4YK',
 '3D1UCPY6GG3UWAQ06K78SDHOHJR83P',
 '3B9J25CZ2570SU1SDBOE27LJZQBCSR',
 '3TD33TP5DLWDITNSIB6S6NLVVZ4ABM',
 '32TMVRKDGNSFE6946YCR2I9UVJN84A',
 '3KWGG5KP6JWXUZONTESSQYMD71TMCP',
 '3QREJ3J433RV7FIQLF8BE4KWC5CKLY']

In [103]:
sorted(requester.database["production"].keys())

['3E22YV8GG10M66IRAKTYNDAKBAHPN5', '3PUV2Q8SV4YBG0Q42LXUWOT1G20DBQ']

In [179]:
for hit_id in hit_ids:
    print(requester.get_url_to_view_hit(hit_id))

https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8
https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8
https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8
https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8
https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8
https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8
https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8


In [219]:
# requester.update_database_with_responses()

In [207]:
# for hit_id, v in requester.database["production"].items():
#     print(v)

In [60]:
urls = []
for ending in sorted(os.listdir("/shared/ethanweber/anno/static/data/responses/")):
    e = ending.replace(".json", "")
    urls.append(f"https://biasgan.ethanweber.me/responses/{e}")
urls = sorted(urls)
# urls
goat.write_to_json("responses_urls.json", urls)

In [70]:
# urls

In [61]:
len(urls)

1422

In [208]:
# responses

In [56]:
requester.update_database_with_responses()

100%|██████████| 474/474 [01:16<00:00,  6.18it/s]


In [53]:
# responses = requester.get_responses_from_database()
# requester.save_all_responses_to_files(responses)

100%|██████████| 341/341 [00:02<00:00, 144.20it/s]


In [71]:
responses = requester.get_responses_from_database()
# requester.save_all_responses_to_files(responses)

In [74]:
# responses.keys()

In [82]:
# for hit_id in responses.keys():
#     t = responses[hit_id]
#     for h in t:
#         print(h["GLOBAL_CONFIG_NAME"])
# #     break

In [59]:
len(responses)

474

In [334]:
# responses["3P888QFVX3OPO3JIBC80270Z6IVOQ1"][0]["GLOBAL_CONFIG"]["QUERY_EXAMPLES"]

In [335]:
# responses["3P888QFVX3OPO3JIBC80270Z6IVOQ1"][0]["QUERY_EXAMPLES_RESPONSES"]

In [ ]:
response["QUERY_EXAMPLES_RESPONSES"] # the answers
response["GLOBAL_CONFIG"]["QUERY_EXAMPLES"] # the questions

In [227]:
# we need some time estimates for each of the tasks...


In [68]:
times = []
for hit_id in tqdm(responses.keys()):
    d = responses[hit_id]
    for hit in d:
#         if "Task3" in hit["GLOBAL_CONFIG_NAME"]:
#             times.append(hit["GLOBAL_REAL_TEST_TIME"])
        times.append(hit["GLOBAL_REAL_TEST_TIME"])
times
print(np.mean(times).astype("int"))

100%|██████████| 474/474 [00:00<00:00, 163535.42it/s]

310


In [67]:
310 / 60

5.166666666666667

In [130]:
# responses["3E22YV8GG10M66IRAKTYNDAKBAHPN5"][0]["GLOBAL_REAL_TEST_TIME"]

In [108]:
# for hit_id in hit_ids:
#     assignments_list = requester.client.list_assignments_for_hit(
#                     HITId=hit_id,
#                     MaxResults=100
#                 )
#     print(assignments_list)
#     print()

In [313]:
# for hit_id in hit_ids:
#     print(requester.client.get_hit(HITId=hit_id))

In [86]:
# requester.get_mean_time_from_resonses(responses)

In [244]:
# stop hits
import datetime
for hit_id in hit_ids:
#     print('HITId:', hit_id)
    # Get HIT status
    status = requester.client.get_hit(HITId=hit_id)['HIT']['HITStatus']
    print('HITStatus:', status)
    # If HIT is active then set it to expire immediately
    if status=='Assignable':
        response = requester.client.update_expiration_for_hit(
            HITId=hit_id,
            ExpireAt=datetime.datetime(2015, 1, 1)
        )
        print(hit_id)

In [97]:
# check the number of unique works for our tasks...
# workerids = []
# for idx, url in enumerate(sorted(urls)):
#     if idx % 3 == 0:
#         print()
#     workerid = url.split("-")[-1]
#     print(workerid)
#     workerids.append(workerid)

In [99]:
# workerids

In [91]:
len(set(workerids))

59

In [92]:
len(urls)

1422